# Analysis of free exploration trajectories in VR
<br>
This notebook contains clean code to preprocess, visualize, and analyze free roaming navigation data. The analyses included in this notebook are experienced integration, and roaming entropy, as well as mean squared displacement. <br>
The set up assumes that there are 4 columns in the navigation logs (x, y, z, heading). The sampling rate in the present dataset is 10 Hz. <br>
For now, we have only included 5 participants' navigation logs to help with the speed of processing.

## Contents
1. [Navigation log preprocessing](#preprocessing)
2. [Trajectory visualization](#trajectories)
3. [Experienced integration](#integration)
4. [Roaming entropy](#entropy)
5. [Mean squared displacement](#msd)

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from scipy import spatial
from matplotlib.colors import ListedColormap
import seaborn as sns

plt.rcParams['figure.figsize'] = [10, 6]

## Navigation log preprocessing <a name="preprocessing"></a>
First, the navigation logs are cleaned and preprocessed for further analyses. Any periods where the participants were stationary for more than 30 seconds are removed.

In [ ]:
### Preprocessing ####
path = r'.\navlogs' # subfolder with nav logs
all_files = glob.glob(path + "\\0*.txt")

# initialize empty list
data = []

for csv in all_files:
    # initialize counter to remove periods of > 30s
    j = 0
    frame = pd.read_csv(csv, sep=";", names=['x', 'y', 'z', 'heading'], header=None)
    total_dur = len(frame)
    print('currently preprocessing', os.path.basename(csv))
    # flip coordinates to make the map upright
    frame.x = -1*frame.x
    frame.z = -1*frame.z
    # store raw row indices
    frame['row_num'] = frame.index
    frame['chunk_index'] = ""
    # calculate timepoint-to-timepoint displacement to identify stationary periods
    r = np.sqrt(frame.x**2 + frame.z**2)
    frame['diff'] = r.diff(-1) 
    # now loop over rows & append a label to each chunk
    # iterrows is slow but it works
    for index, row in frame.iterrows():
        if (frame.loc[index,'diff'] == 0 and frame.loc[index+1, 'diff'] == 0):
            frame.loc[index,'chunk_index'] = j
        else: 
            j = j + 1
            frame.loc[index,'chunk_index'] = "mov"
    # add count for each data chunk
    frame['count'] = frame.groupby('chunk_index')['chunk_index'].transform('count') 
    # keep only chunks where participants were moving, or where the stationary count is >= 30 seconds
    frame = frame[(frame['chunk_index'] == "mov") | (frame['count'] < 300)]
   
    print('done with', os.path.basename(csv))
    
    # save clean nav log as csv for future use
    filename = os.path.basename(csv)
    sub_id = filename.split('_', 1)[0]
    frame['sub'] = sub_id
    path=r'.\navlogs_clean\\'
    
    csv_filename = path + sub_id + '_clean_navlog.csv'
    frame.to_csv(csv_filename)
    
    # store each dataset in the list for the next step
    data.append(frame)

print('*** done preprocessing nav logs ***')

## Trajectory visualization <a name="trajectories"></a>

We can now display all of the participants' navigation trajectories in Virtual Silcton. To adapt this to another data set, you need to read in your image of the environment and transform the x & z coordinates to match the image. This transformation step is for <i> visualization only</i>. It does not affect any of the exploration measures, so it's not a problem if the image is slightly misaligned.

In [ ]:
## uncomment if you need to read in again ##
# path = r'.\navlogs_clean'
# all_files = glob.glob(path + "\\00*.csv")

# data=[]
# for csv in all_files:
#    frame = pd.read_csv(csv, sep=",")
#    data.append(frame)

bigframe = pd.concat(data)
bigframe = bigframe.dropna() 
x = pd.to_numeric(bigframe['x'])
z = pd.to_numeric(bigframe['z'])

# match image coordinates
x= x + 738
z= z + 508

im = plt.imread("Full_Info_Silcton_Map.png")
implot = plt.imshow(im, extent = [0, 1450, 170, 1030])
plt.scatter(x, z, c = bigframe['heading'], s = 8)

plt.show()

## Experienced integration <a name="integration"></a>

Now that the navigation logs are preprocessed, we can look at some exploration measures. <br> First, we will relate participants' navigation tendencies to a measure of space syntax called <i>axial integration</i>. This measure expresses each segment in the environment in terms of its topological proximity to every other segment. Higher values therefore reflect higher integration with the rest of the environment. <br>
In our analysis of exploration patterns, we will apply the corresponding integration value to each coordinate in space traversed by each participant. We will only keep coordinates that are < 10 virtual meters away from the path.

In [ ]:
## Plot integration map & overlay trajectories ##
map_file = pd.read_csv("VGA_integration_populated.txt", sep="\t")

path = r'.\navlogs_clean'
all_files = glob.glob(path + "\\*.csv")

data=[]
for csv in all_files:
    frame = pd.read_csv(csv, sep=",")
    data.append(frame)

bigframe = pd.concat(data)
bigframe = bigframe.dropna() 
x = pd.to_numeric(bigframe['x'])
z = pd.to_numeric(bigframe['z'])

cmap = ListedColormap(sns.color_palette("viridis", 20))   
plt.scatter(map_file.x, map_file.y, c = map_file.integ, cmap=cmap, s = 5)
plt.scatter(x, z, c = 'grey', s = 5, alpha = .1)

plt.show()

Now, the idea is to extract the corresponding integration value from the map & create a timeseries for each coordinate each participant traversed in the environment:

In [ ]:
# read in map file 
map_file = pd.read_csv("VGA_integration_populated.txt", sep="\t")

# save as array to match coordinates
map_arr = map_file[['x', 'y']].to_numpy()

path = r'.\navlogs_clean'
all_files = glob.glob(path + "\\*.csv")

data=[]
for csv in all_files:
    frame = pd.read_csv(csv, sep=",")
    data.append(frame)

# initialize empty list
data_integ = []

for frame in data:
    # append total nav duration
    frame['total_dur'] = len(frame)
    print('currently processing', frame['sub'].iloc[0])
    # initialize columns with distance from integration map & integration values
    frame['distance_coord'] = ""
    frame['conn'] = ""
    # go through rows, append the integration value & distance from map
    # !! this step takes a while !! this is because it iterates over rows.
    # there must be a smarter way to do this, but this was the only way that I ended up finding that worked
    # spatial.kdtree works by finding the nearest point to each pair of x,z coordinates
    for index, row in frame.iterrows():
        distance,map_index = spatial.KDTree(map_arr).query([frame.x[index], frame.z[index]])
        frame.loc[index, 'distance_coord'] = distance
        frame.loc[index, 'integ'] = map_file['integ'][map_index]
    # remove points where participants went off-path by more than 10 vm
    frame = frame[frame['distance_coord'] < 10] 
    # append duration spent on path (> 10 vm off-path)
    dur_on_path = len(frame)
    frame['dur_on_path'] = dur_on_path
    
    data_integ.append(frame)
    
print('*** done with experienced integration analysis ***')

In [ ]:
# concatenate into one dataframe
# uncomment below to write to csv
integ_frame = pd.concat(data_integ, ignore_index = False) 
#integ_frame.to_csv('silcton_freeexplore_integ_vals_clean.csv')

# summarise
# uncomment below to write to csv
integ_frame['integ'] = integ_frame['integ'].astype(float)
mean_integ = integ_frame.groupby(['sub']).agg({'integ': 'mean'})
#mean_integ.to_csv('silcton_freeexplore_integ_per_sub.csv')

As a final step, to make it more intuitive, we can plot the raw trajectories + the trajectories 'weighted' by their integration value. This also shows where the integration values got filtered out off-path. 

In [ ]:
# initialize interactive seaborn color picker
my_palette = sns.choose_cubehelix_palette()
# parameters to match published color palette:
# n_colors = 15
# start = 1.10
# rot = -1.00
# gamma = 0.70
# hue = 1.00
# light = 0.70
# dark = 0.10

In [ ]:
# convert to cmap format
cmap = ListedColormap(my_palette)

In [ ]:
## Visualize overlap between integration and trajectories ##

# read in overall frame with all data
connframe = pd.read_csv("silcton_freeexplore_integ_vals_clean.csv", sep=",")

# match image coordinates
connframe.x = connframe.x + 738
connframe.z = connframe.z + 510

path = r'.\navlogs_clean'
all_files = glob.glob(path + "\\*.csv")

data=[]
for csv in all_files:
    frame = pd.read_csv(csv, sep=",")
    data.append(frame)

rawframe = pd.concat(data)
rawframe = rawframe.dropna() 
x = pd.to_numeric(rawframe['x'])
z = pd.to_numeric(rawframe['z'])

# match image coordinates
x = x + 738
z = z + 510

im = plt.imread("Full_Info_Silcton_Map.png")
implot = plt.imshow(im, extent = [0, 1456, 170, 1032], alpha = .5)

plt.scatter(x, z, c = "grey", alpha = .05, s = .5)
plt.scatter(connframe.x, connframe.z, c = connframe['integ'], cmap = cmap, alpha = 1, s = 1) 
plt.axis('off')
plt.show()
# uncomment below to save figure
#plt.savefig('integration_x_traces.png', bbox_inches = 'tight')

## Roaming entropy <a name="entropy"></a>
As a second exploration measure, let's calculate each participant's roaming entropy. This translates to the coverage of the environment. Participants with low roaming entropy cover fewer states relative to those with high roaming entropy. In the present analysis, we divided the environment into 50 x 50 states. This can easily be adjusted by changing the number in the 'pd_cut' command below.

In [ ]:
### Roaming Entropy ###
## read in clean datasets
path = r'.\navlogs_clean'
all_files = glob.glob(path + "\\*.csv")

data=[]
for csv in all_files:
    frame = pd.read_csv(csv, sep=",")
    data.append(frame)

# initialize empty frames
data_states = []

# in this loop, we'll first count up the number of timepoints each participant spent in each state
for frame in data:
    print('calculating state exploration for', frame['sub'].iloc[0])
    # split into 50 x 50 grid
    frame['x_cut_50'] = pd.cut(frame.x, np.linspace(-738, 738, 51), right=False)
    frame['y_cut_50'] = pd.cut(frame.z, np.linspace(-523, 523, 51), right=False)
    # example: division into 20 x 20 grid
    #frame['x_cut_20'] = pd.cut(frame.x, np.linspace(-738, 738, 21), right=False)
    #frame['y_cut_20'] = pd.cut(frame.z, np.linspace(-523, 523, 21), right=False)
    frame['count_events'] = ""
    count_frame = frame.groupby(['x_cut_50', 'y_cut_50']).agg({'count_events': ['count']})
    count_frame['sub'] = frame['sub'].iloc[0]
    data_states.append(count_frame)

print('*** done with state exploration ***')
states_frame = pd.concat(data_states, ignore_index=False) 
states_frame.reset_index(inplace=True)
states_frame.columns = ['x_cut', 'y_cut', 'count', 'sub']
# uncomment to write to csv
#states_frame.to_csv('silcton_freeexplore_50_states_count.csv')

# now, we'll calculate roaming entropy for each participant
cur_dim = states_frame.groupby(['sub']).size()
cur_dim = cur_dim.iloc[0]

def roaming_entropy(event_count):
    event_count = event_count[event_count != 0]
    norm_counts = event_count/np.sum(event_count)    
    return -(norm_counts * np.log2(norm_counts)/np.log(cur_dim)).sum()

entropy_frame = states_frame.groupby('sub').apply(lambda x: roaming_entropy(x['count']))

# uncomment to write to csv
#entropy_frame.to_csv("silcton_roaming_entropy_per_sub.csv")
print('*** done with entropy calculation ***')

## Mean squared displacement <a name="msd"></a>

The final measure we included in our overall model is Mean Squared Displacement. It essentially translates to <i>speed</i> of movement in the environment. However, it does not necessarily relate to coverage; a participant can move a lot in a small area of the environment. 

In [ ]:
# initialize empty lists
data_msd = []
data_diff_sq = []

path = r'.\navlogs_clean'
all_files = glob.glob(path + "\\*clean_navlog.csv")

data=[]
for csv in all_files:
    frame = pd.read_csv(csv, sep=",")
    data.append(frame)

for frame in data:
    print('calculating MSD for', frame['sub'].iloc[0])
    # we have already calculated the 'diff' column earlier to detect periods of movement
    # now, all we need to do is to square that value & average across entire timecourse to calculate mean square displacement
    diff_sq = frame['diff']**2
    MSD = np.mean(diff_sq)
    data_msd.append(MSD)
    
    ## to calculate MSD at longer lags:
    ## uncomment below and change the value in the 'r.diff' operation, e.g. -2, -3 ...
    #r = np.sqrt(frame.x**2 + frame.z**2)
    #frame['diff'] = r.diff(-2) 
    #diff_sq = frame['diff']**2
    #MSD = np.mean(diff_sq)
    #data_msd.append(MSD)
    
    # create a new dataframe with moment-by-moment values for later use
    diff_dset = pd.DataFrame(diff_sq)
    sub_id = frame['sub'].iloc[0]
    diff_dset['sub'] = sub_id
    diff_dset['row_num'] = frame['row_num']
    data_diff_sq.append(diff_dset)

print('*** done with MSD calculation ***')

# uncomment below to save dataframes to csv
diff_sq_frame = pd.concat(data_diff_sq, ignore_index=False) 
#diff_sq_frame.to_csv('silcton_diff_sq_timepoints_lag1.csv')
data_msd = pd.DataFrame(data_msd)
#data_msd.to_csv("silcton_msd_lag1_per_sub.csv")

In [ ]:
## Heatmap of all trajectories
## 50 x 50 grid as in the entropy calculations

path = r'.\navlogs_clean'
all_files = glob.glob(path + "\\0*.csv")

data=[]
for csv in all_files:
    frame = pd.read_csv(csv, sep=",")
    data.append(frame)

bigframe = pd.concat(data)
bigframe = bigframe.dropna() 

x = pd.to_numeric(bigframe['x'])
z = pd.to_numeric(bigframe['z'])

# adjust coordinates to overlap with the map image
x = x + 750
z = z + 510

im = plt.imread("Full_Info_Silcton_Map.png")
plt.imshow(im, extent = [0, 1460, 160, 1038], alpha = .5, aspect = 'auto')

plt.hist2d(x, z, bins=(50, 50), cmin = .1, range = [[0, 1500], [0, 1038]], cmap = "Spectral_r", alpha = .85) 
plt.colorbar()
plt.clim(0,600)

plt.axis('off')
plt.show()

# uncomment to save figure
#plt.savefig('heatmap_50x50.png', bbox_inches='tight')

Plot locations where participants stopped and visually scanned their environment:


In [ ]:
# read in map file 
map_file = pd.read_csv("VGA_integration_populated.txt", sep="\t")

map_file.x = map_file.x + 738
map_file.y = map_file.y + 510

# save as array to match coordinates
map_arr = map_file[['x', 'y']].to_numpy()

frame = pd.read_csv('silcton_detected_scanning_events_clean.csv', sep=",")

plt.scatter(frame.x_pos, frame.z_pos)